In [1]:
import pandas
import mysql.connector
import pandas 
import numpy as np

import plotly.express as px
import plotly.graph_objects as go


In [2]:
# Connection à la base

conn = mysql.connector.connect(
    host="192.168.71.42",
    database="myImagiX",
    user="root",
    password="fskfes" )


In [3]:
def read_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute( query )
        names = [ x[0] for x in cursor.description]
        rows = cursor.fetchall()
        return pandas.DataFrame( rows, columns=names)
    finally:
        if cursor is not None:
            cursor.close() 


# Piscine d'Amaury

In [8]:
qr = "SELECT date,state,stateLabel FROM `box_events` LEFT JOIN event_types ON event_types.state = box_events.paramNew WHERE box_events.box_id = 182 AND box_events.date >= '2022-05-01 00:00:00' and event_types.id = box_events.eventId AND idLabel LIKE 'Changement d’état de la filtration' ORDER BY `box_events`.`date` ASC "

dff=read_query(conn,qr)
dff.head(6)

,date,state,stateLabel
0,2022-05-01 03:31:04,2,2 Marche manuel ON
1,2022-05-01 09:25:53,-2,-2 Arrêt manuel OFF
2,2022-05-01 09:29:06,2,2 Marche manuel ON
3,2022-05-01 09:44:29,-2,-2 Arrêt manuel OFF
4,2022-05-01 09:49:09,2,2 Marche manuel ON
5,2022-05-01 18:57:35,0,0 Arrêt AUTO


In [10]:

#normalisation de la colonne state
def transform(x,a,b):
    
    return ((x - np.min(x))/(np.max(x)-np.min(x))) * (b-a) + a

dff['normalised_state']= transform(dff['state'],0,50)

dff.head(6)



,date,state,stateLabel,normalised_state
0,2022-05-01 03:31:04,2,2 Marche manuel ON,32.142857
1,2022-05-01 09:25:53,-2,-2 Arrêt manuel OFF,17.857143
2,2022-05-01 09:29:06,2,2 Marche manuel ON,32.142857
3,2022-05-01 09:44:29,-2,-2 Arrêt manuel OFF,17.857143
4,2022-05-01 09:49:09,2,2 Marche manuel ON,32.142857
5,2022-05-01 18:57:35,0,0 Arrêt AUTO,25.000000


In [11]:

qry0 = "SELECT * FROM `box_stats` WHERE type LIKE 'dirtying' and box_id= (SELECT id FROM boxes WHERE serialNumber LIKE '%37cf0') ORDER BY `box_stats`.`date` ASC "
df = read_query(conn,qry0)
df0 =read_query(conn,qry0)

fig = px.line(df0, x="date", y="value", color="type" )
fig.add_scatter(x=dff.date, y=dff['normalised_state'], mode='lines',name="filtration",marker_color='rgba(0, 0, 0, .9)')
#fig.add_scatter(x=df_dirtying.date, y=df_dirtying['threshold'], mode='lines',name="seuil",marker_color='rgba(255, 0, 0, .9)')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True  
        ),
        type="date"
    )
) 

fig.show()


In [20]:
fig.write_html('encrassement.html')

In [26]:
from tracemalloc import take_snapshot


THRESHOLD = 10
qry0 = "SELECT * FROM `box_stats` WHERE type IN ('dirtying') and box_id= (SELECT id FROM boxes WHERE serialNumber LIKE '%37cf0') ORDER BY `box_stats`.`date` ASC "
df = read_query(conn,qry0)
df0 =read_query(conn,qry0)
df_dirtying = df.iloc[:,3:5]

df_dirtying['threshold'] = THRESHOLD
df_dirtying['event'] = df_dirtying.value < df_dirtying.threshold
df_dirtying['event'] = np.where(df_dirtying['event']==True, 1 ,0)
#df_dirtying = df_dirtying.set_index('date')
df_dirtying


,date,value,threshold,event
0,2022-04-28 22:59:23,12.0,10,0
1,2022-04-30 13:14:22,12.0,10,0
2,2022-05-01 08:14:24,12.0,10,0
3,2022-05-02 15:00:00,12.0,10,0
4,2022-05-03 15:00:00,12.0,10,0
...,...,...,...,...
89,2022-09-12 07:00:56,14.0,10,0
90,2022-09-12 16:00:00,14.0,10,0
91,2022-09-13 16:00:00,14.0,10,0
92,2022-09-14 14:31:19,14.0,10,0


In [28]:
import plotly.express as px


fig = px.bar(df_dirtying, x=df_dirtying.date, y="value")
fig.add_scatter(x=df_dirtying.date, y=df_dirtying['value'], mode='lines',name="values",marker_color='rgba(0, 0, 0, .9)')
fig.add_scatter(x=df_dirtying.date, y=df_dirtying['threshold'], mode='lines',name="seuil",marker_color='rgba(255, 0, 0, .9)')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()



In [15]:
from doctest import master
from email.errors import MissingHeaderBodySeparatorDefect
from logging import lastResort


df_dirtying_true = df_dirtying[df_dirtying.event == 1]
df_dirtying_true.head(5)


,date,value,threshold,event
10,2022-05-09 16:59:39,3.0,10,1
11,2022-05-10 09:59:37,3.0,10,1
12,2022-05-11 09:29:36,3.0,10,1
13,2022-05-12 07:29:39,3.0,10,1
14,2022-05-13 15:29:42,3.0,10,1
15,2022-05-14 17:00:00,3.0,10,1
16,2022-05-16 13:59:43,7.0,10,1
17,2022-05-16 17:00:00,7.0,10,1
21,2022-06-30 16:00:35,6.0,10,1
22,2022-07-01 15:30:32,6.0,10,1


In [87]:
import plotly.express as px

fig = px.line(df_dirtying, x=df_dirtying.date, y=df_dirtying.value) 
fig.add_scatter(x=df_dirtying_true.date, y=df_dirtying_true.value, mode='markers',name="nettoyage de cartouche")
fig.add_scatter(x=df_dirtying.date, y=df_dirtying['threshold'], mode='lines',name="seuil",marker_color='rgba(20, 200, 0, .9)')
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
         
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()


